## Notebook to pull all regional sites from the 2022 MHSAA Baseball tournement






In [1]:
from bs4 import BeautifulSoup
import requests



d1_link = 'https://my.mhsaa.com/Sports/MHSAA-Tournament-Brackets/BracketGroup/9/SportSeasonId/421324/Classification/1'
d2_link = 'https://my.mhsaa.com/Sports/MHSAA-Tournament-Brackets/BracketGroup/9/SportSeasonId/421324/Classification/2'
d3_link = 'https://my.mhsaa.com/Sports/MHSAA-Tournament-Brackets/BracketGroup/9/SportSeasonId/421324/Classification/3'
d4_link = 'https://my.mhsaa.com/Sports/MHSAA-Tournament-Brackets/BracketGroup/9/SportSeasonId/421324/Classification/4'

divs = [d1_link, d2_link, d3_link, d4_link]



In [2]:
#thirdLevelContests > div > div.contestboxhover > div.contestinfo > span.contestlocation >


# use python and beautiful soup to parse all of the the location names and map links from the span.contestlocation tag in this url https://my.mhsaa.com/Sports/MHSAA-Tournament-Brackets/BracketGroup/9/SportSeasonId/421324/Classification/1

# return the results as a pandas dataframe

import requests
from bs4 import BeautifulSoup
import pandas as pd

i = 4

# Request HTML
url = f'https://my.mhsaa.com/Sports/MHSAA-Tournament-Brackets/BracketGroup/9/SportSeasonId/421324/Classification/{i}'
html = requests.get(url).content

# Parse HTML
soup = BeautifulSoup(html, 'html.parser')

# Find all location tags
locations = soup.find_all('span', {'class': 'contestlocation'})

# Pull out location and map link
data = []
for location in locations:
    loc = location.text.strip()
    link = location.find('a')['href']
    div = 'D'
    data.append([loc, link, div])

# Create DataFrame
df_4 = pd.DataFrame(data, columns=['Location', 'Map Link', 'Division'])

####### DOING IT DIRTY TO JUST GET THE DATA I WANT ########


In [3]:
df_1

NameError: name 'df_1' is not defined

In [ ]:
#### Combine all the dataframes into one

df = pd.concat([df_1, df_2, df_3, df_4])

df.info()


# drop duplicates

df.drop_duplicates(inplace=True)
df.info()

# ### Outpu to csv

df.to_csv('data/school_info//regional_locations_all.csv', index=False)

## Working for basic columns, 
## Want to make a column to show the division




In [5]:
import pandas as pd


mhsaa_file = 'data/tables/area_distance.csv'

regional_file = 'data/school_info//regional_locations_all.csv'

plot_df = pd.read_csv(mhsaa_file)

regional_df = pd.read_csv(regional_file)



### Clean up some of the school names

regional_df['Location'] = regional_df['Location'].str.replace('High School', 'HS')
regional_df['Location'] = regional_df['Location'].str.replace('MAP','')
regional_df['Location'] = regional_df['Location'].str.replace('-', '')
regional_df['Location'] = regional_df['Location'].str.replace('Baseball', '')
plot_df.info()
regional_df.info()

### Use fuzzywuzzy to match the school names

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# school names from mhsaa
plot_schools = plot_df['field'].tolist()

# field names from regional locations
regional_schools = regional_df['Location'].tolist()

## Empty list for results
matches = []

for i in range(len(regional_schools)):
    mat = process.extract(regional_schools[i], plot_schools, limit=1)

    matches.append(mat)


## Split the list of tuples into two lists

school_names = []
scores = []

for i in range(len(matches)):
    school_names.append(matches[i][0][0])
    scores.append(matches[i][0][1])


## Add the matches to the regional dataframe

regional_df['plot_match'] = school_names
regional_df['score'] = scores

regional_df.head()
# match the school names


# matches
# print(regional_schools)

### Output csv for check

# regional_df.to_csv('TEMP/regional_match_check.csv', index=False)
#



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 646 entries, 0 to 645
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       646 non-null    int64  
 1   field            646 non-null    object 
 2   level            646 non-null    object 
 3   size_factor      646 non-null    int64  
 4   foul_ft          646 non-null    float64
 5   fop_ft           646 non-null    float64
 6   foul_pct         646 non-null    float64
 7   h_lon            646 non-null    float64
 8   h_lat            646 non-null    float64
 9   home_plate       646 non-null    object 
 10  min_distance     646 non-null    float64
 11  max_distance     646 non-null    float64
 12  avg_distance     646 non-null    float64
 13  median_distance  646 non-null    float64
 14  match            490 non-null    object 
 15  school_id        490 non-null    float64
 16  id               646 non-null    object 
dtypes: float64(10), 

,Location,Map Link,Division,plot_match,score
0,East Lansing HS,http://maps.google.com/maps?q=MacDonald+Middle...,A,East Lansing HS,100
1,Northville HS,http://maps.google.com/maps?q=Northville+High+...,A,Northville HS,100
2,Lincoln Ypsilanti,http://maps.google.com/maps?q=Lincoln+Baseball...,A,Ypsilanti - Lincoln HS,95
3,Coldwater HS (Coldwater @ Borden Field),http://maps.google.com/maps?q=Coldwater+High+S...,A,Coldwater HS,90
4,Borden Field Coldwater,http://maps.google.com/maps?q=Borden+Field++Co...,A,Coldwater HS,86


In [7]:

### Output csv for check

regional_df.to_csv('TEMP/regional_match_check.csv', index=False)


### Good Matches
# Need to add coulmns tot he plot book with region_host and regional_div columns

